## Введение

Часто при реализации алгоритмов вычислительной геометрии возникают проблемы, связанные с точностью вычисления арифметических выражений. Погрешности, возникающие из-за ограниченной точности арифметики с плавающей точкой, приводят к нарушению инвариантов алгоритма и, как следствие, неверному результату. Для устранения этих проблем используют фильтрованные вычисления. Для начала все вычисления проводят в арифметике с плавающей точкой, при недостаточной точности переходят к [интервальной арифметике](../5_affine_space/foundation.ipynb ), а затем используют рациональную арифметику. Однако рациональная арифметика имеет свои недостатки. Во-первых, даже эффективные ее реализации сложны и медленны. Во-вторых, она требует или сложной разработки, или добавления зависимости от сторонних библиотек. Поэтому на практике удобнее использовать **адаптивную арифметику** (*англ. adaptive precision arithmetic*).

В качестве источника информации для данной статьи использована статья [J.R.Shewchuk "Adaptive Precision Floating-Point Arithmetic and Fast Robust Geometric Predicates"](robustr.pdf).

## Основные понятия

Согласно стандарту IEEE 754 процессоры хранят числа с плавающей точкой в форме $ \pm \mathrm{significand} \times 2^{\mathrm{exp}}$. В записи двоичных чисел будет использоваться *двоичная* точка. Значащая часть числа (мантисса) представляется в виде $\mathrm{p}$-битного двоичного числа в форме $\mathrm{b.bbb} \dots$, где каждое $\mathrm{b}$ обозначает один бит. Еще один бит отводится под знак.

Точные арифметические операции могут потребовать для сохранения значения больше чем $\mathrm{p}$-бит. Каждое такое число с произвольной точностью (*англ. arbitrary precision value*) представляется в виде **разложения** (*англ. expansion*) $x = x_n + \dots + x_1 + x_0$, где каждое $x_i$ является компонентой числа $x$ и представляется в виде $\mathrm{p}$-битного числа с плавающей точкой.

Два числа с плавающей точкой $x$ и $y$ называются **неперекрывающимися** (*англ. nonoverlapping*), если номер наименьшего значимого бита числа $x$ (нумерация справа налево) больше, чем номер наибольшего значимого бита числа $y$, или наоборот. Более формально, $x$ и $y$ не перекрываются, если существуют такие целые числа $r$ и $s$, что $x = r\times2^s$ и $|y| < 2^s$, или $y = r\times2^s$ и $|x| < 2^s$. Ноль не перекрывается ни с одним другим числом.

Например, числа $1100$ и $-10.1$ не перекрываются, а $110$ и $10$ — перекрываются.

Разложение является неперекрывающимся, если все его компоненты взаимно не перекрываются. В дальнейшем будут рассматриваться неперекрывающиеся разложения, упорядоченные по убыванию компонент (то есть $x_n$ — наибольшая, $x_0$ — наименьшая). Такие разложения позволяют легко определить знак числа (берется знак наибольшей компоненты) и используются для грубого приближения значения (берется наибольшая компонента).

Два числа с плавающей точкой  $x$ и $y$ называются **смежными** (*англ. adjacent*), если они перекрываются, если $x$ и $2y$ перекрываются, или $2x$ и $y$ перекрываются.

Например, числа $1100$ и $11$ смежные, а $1000$ и $11$ — нет.

Разложение является несмежным, если никакие две его компоненты не смежны.

Концептуальным отличием между алгоритмами рациональной арифметики и многокомпонентной арифметики является тот факт, что последние позволяют случаться ошибкам округления, а затем принимают их во внимание. Чтобы находить ошибки округления быстро и правильно, необходимо наложить определенные ограничения на операции процессора над числами с плавающей точкой. 

Все алгоритмы предполагают, что сложение, вычитание и умножение выполняются с **точным округлением** (*англ. exact rounding*). Это значит, что если точный результат может быть представлен в $p$ битах, то результатом округления будет точное значение числа. Иначе, результатом округления будет ближайшее $p$-битное значение. Если результат попадает точно посередине между двумя соседними $p$-битными значениями, то существует два варианта округления.

**Округление до ближайшего четного** (*англ. round-to-even*) — правило, при котором округление производится к ближайшему $p$-битному четному значению.


**Округление к нулю** (*англ. round-toward-zero*) — правило, при котором округление производится к ближайшему $p$-битному значению, находящемуся между точным значением и нулем.

Стандарт IEEE 754 использует округление до ближайшего четного по умолчанию.

Далее в этой статье символами $\oplus, \ominus$ и $\otimes$ будут обозначаться $p$-битные сложение, вычитание и умножение с точным округлением соответственно.

Из-за округления данные операции теряют некоторые важные свойства, например, ассоциативность: $(1000 \oplus 0.011) \oplus 0.011 = 1000$, но $1000 \oplus(0.011 \oplus 0.011) = 1001$.

При анализе округления часто используют оператор $\mathrm{ulp}$ (*англ. unit in the last place*) — эффективную величину самого младшего $p$-ого бита. Он определяется для конкретного числа: $\mathrm{ulp}(x) = 2^{-p+1+\mathrm{exp}}$. Например, в четырехбитной арифметике $\mathrm{ulp}(-1100) = 1$, а $\mathrm{ulp}(1) = 0.001$.

Также используют оператор $\mathrm{err}(a\odot b)$, который показывает ошибку округления результата выполнения операции $\odot$. Необходимо отметить, что $\mathrm{ulp}$ беззнаковая величина, а $\mathrm{err}$ всегда имеет знак. Для операций сложения, вычитания и умножения с точным округлением выполняется равенство:

$$ a\odot b = a\cdot b  + \mathrm{err}(a\odot b), где   \left | \mathrm{err}(a\odot b) \right | \leqslant \frac{1}{2}\mathrm{ulp}(a\odot b)$$

## Свойства двоичной арифметики

Иногда можно найти более точные границы для ошибки округления. Лемма 1 используется, когда один операнд намного меньше другого, а лемма 2 — когда сумма операндов близка к степени двойки. В утверждениях 1—5, $a$ и $b$ — $p$-битные числа с плавающей точкой.

> **Лемма 1** Пусть $a \oplus b = a + b + \mathrm{err} (a \oplus b) $.
Тогда ошибка округления $\mathrm{err}(a \oplus b)$ не больше чем $\left|a\right|$ или $\left|b\right|$. (Аналогично для вычитания)

$\triangleright$
<div style="padding-left:50px">
Не уменьшая общности предположим, что $|a| \geqslant |b|$. Сумма $a \oplus b $ — это $p$-битное число с плавающей точкой, ближайшее к $a + b$. Но $a$ — это $p$-битное число с плавающей точкой. Если $\mathrm{err}(a \oplus b) > |b|$, то в качестве ответа можно было взять число $a$, и ошибка была бы равна $|b|$. Значит $\mathrm{err}(a \oplus b) \leqslant |b| \leqslant |a|$.
</div>
$\triangleleft$

> **Следствие 2** Ошибка округления $\mathrm{err}(a \oplus b)$ может быть представлена в $p$ битах

$\triangleright$
<div style="padding-left:50px">
Не уменьшая общности предположим, что $ \left|a\right|\geqslant \left|b\right|$. Наименьший ненулевой бит $\mathrm{err}(a \oplus b)$ по величине не меньше, чем $\mathrm{ulp}(b)$. По лемме 1, $\mathrm{err}(a \oplus b) \leqslant \left|b\right|$. Значит, мантисса $\mathrm{err}(a \oplus b)$ не длиннее, чем $p$ бит. Следовательно, ошибка округления $\mathrm{err}(a \oplus b)$ может быть представлена в $p$ битах.
</div>
$\triangleleft$

> **Лемма 3** Пусть $a \odot b = a \cdot b + \mathrm{err} (a \odot b) $.
Тогда 

>a) Если $\left|\mathrm{err} (a \odot b)\right| \geqslant 2^i$ для целого $i$, то $\left|a\cdot b\right| \geqslant 2^i(2^p+1)$

>б) Если $\left|\mathrm{err} (a \odot b)\right| > 2^i$ для целого $i$, то $\left|a\cdot b\right| > 2^i(2^{p+1}+1)$

$\triangleright$
<div style="padding-left:50px">
а) Каждое из чисел $2^i(2^p),2^i(2^p-1),2^i(2^p-2),\dots ,0$ выражается в $p$ битах. Любое значение $\left|a\cdot b\right|<2^i(2^p-1)$ находится на расстоянии, меньшем чем $2^i$ от одного числа из ряда.

б) Каждое из чисел $2^i(2^{p+1}),2^i(2^{p+1}-2),2^i(2^{p+1}-4),\dots ,0$ выражается в $p$ битах. Любое значение $\left|a\cdot b\right|\leqslant2^i(2^{p+1}+1)$ находится на расстоянии, меньшем чем $2^i$ от одного числа из ряда.
</div>
$\triangleleft$

<center><img src="img/lemma1.png"/></center>
<font size="2"><i><center>Рис. 1</center>Вертикальные линии обозначают 4-битные значения с плавающей точкой. Ошибка округления — расстояние между $a+b$ и $a \oplus b$. Лемма 1 утверждает, что ошибка не может превышать $\left|b\right|$. Лемма 3б) утверждает, что если $\left|(a+b)\right| \leqslant 2^i(2^{p+1}+1)$ (для $i=-2$ и $p=4$ величина $a+b$ попадает на рисунке в выделенный регион), то ошибка не превышает $2^i$. Эта лемма полезна, когда значение близко к степени 2.</i></font>

Две следующие леммы показывают, в каких случаях вычисления компьютера точны. Первая из них утверждает, что сложение и вычитание точны, если величина результата меньше величины операндов.

> **Лемма 4** Пусть $\left|a + b \right| \leqslant \left|a\right|$ и $\left|a + b \right| \leqslant \left|b\right|$. Тогда $a \oplus b = a + b$. (Аналогично для вычитания)

$\triangleright$
<div style="padding-left:50px">
Не уменьшая общности предположим, что $|a| \geqslant |b|$. Наименьший ненулевой бит $a + b$  по величине не меньше, чем  $\mathrm{ulp}(b)$. Однако, $\left|a + b \right| \leqslant \left|b\right|.$ Следовательно, $a+b$ может быть представлена в $p$ битах.
</div>
$\triangleleft$

> **Лемма 5** Пусть $b \in \left [ \frac{a}{2}, 2a \right ]$. Тогда $a \ominus b = a - b$.

$\triangleright$
<div style="padding-left:50px">
Не уменьшая общности предположим, что $|a| \geqslant |b|$. (Другой случай симметричен, так как  $a \ominus b = -b \ominus -a$.) Тогда $b \in \left [ \frac{a}{2}, a \right ]$. Разность удовлетворяет неравенству $\left|a-b\right|\leqslant\left|b\right|\leqslant\left|a\right|$, и требуемый результат следует из Леммы 4.
</div>
$\triangleleft$

<center><img src="img/lemma5.png"/></center>
<font size="2"><i><center>Рис. 2<br/>Примеры к лемме 5</center></i></font>

## Простое суммирование

Важной базовой операцией во всех алгоритмах, основанных на представлении чисел в виде разложений, является сумма двух $p$-битных величин, результатом которой является разложение длиной два. Есть два алгоритма для выполнения этой задачи — алгоритмы Деккера и Кнута.

> **Теорема 6 (Dekker)** Пусть $a$ и $b$ — $p$-битные числа, причем $\left|a\right| \geqslant \left|b\right|$. Тогда следующий алгоритм вернет неперекрывающееся разложение $x + y$ такое, что $a + b = x + y$, где $x$ - приближение суммы $a + b$, а $y$ представляет собой ошибку округления при вычислении $x$.

<div style="padding-left:100px">
Fast-Two-Sum (a, b)<br/>
1. $x \Leftarrow a \oplus b$<br/>
2. $b_{virtual} \Leftarrow x \ominus a$<br/>
3. $y \Leftarrow b \ominus b_{virtual}$<br/>
4. return $(x, y)$<br/>
</div>


$\triangleright$
<div style="padding-left:50px">
В строке 1 вычисляется значение $a + b$, но оно может быть подвержено ошибке округления, поэтому $x = a + b + \mathrm{err}(a\oplus b)$. По условию $\left|a\right| \geqslant \left|b\right|$, следовательно $a$ и $x$ должны быть одного знака (или $x = 0$).

В строке 2 вычисляется значение $b_{virtual}$ — число *реально* прибавленное в строке 1 к $a$. Вычитание выполняется точно, что может быть доказано рассмотрением двух случаев. Если $a$ и $b$ одного знака или $\left|b\right|\leqslant\frac{\left|a\right|}{2}$, то $x \in \left [ \frac{a}{2}, 2a \right ]$ и можно применить лемму 5. Иначе, если $a$ и $b$ имеют разный знак и $\left|b\right|>\frac{\left|a\right|}{2}$, то $b \in \left [ -\frac{a}{2}, -a \right ]$ и лемма 5 может быть применена к строке 1. Это означает, что значение $x$ было вычислено точно и $b_{virtual} = b$. В любом случае, вычитание в строке 2 точное, следовательно $b_{virtual} = x - a = b + \mathrm{err}(a \oplus b)$.

В строке 3 вычитание также точное, что следует из лемм 1 и 4. По следствию 2 значение $b - b_{virtual} = -\mathrm{err}(a\oplus b)$ может быть представлено в $p$ битах.

Следовательно $y = -\mathrm{err}(a\oplus b)$ и $x = a + b + \mathrm{err}(a\oplus b)$, что влечет $a + b = x + y$. Точное округление гарантирует, что $\left|y\right| \leqslant \frac{1}{2}\mathrm{ulp}(x)$, значит $x$ и $y$ не перекрываются.
</div>
$\triangleleft$
<br\>
Корректность алгоритма для вычитания доказывается аналогично.
<div style="padding-left:100px">
Fast-Two-Diff (a, b)<br/>
1. $x \Leftarrow a \ominus b$<br/>
2. $b_{virtual} \Leftarrow a \ominus x$<br/>
3. $y \Leftarrow b_{virtual} \ominus b$<br/>
4. return $(x, y)$<br/>
</div>

<center><img src="img/fts1.png"/></center>
<center><br><font size="2"><i>Рис. 3<br/> Демонстрация алгоритма Fast-Two-Sum, когда $a$ и $b$ имеют один знак. $111100 + 1001 = 1001000 + -11$</i></font></center>

<center><img src="img/fts2.png"/></center>
<center><br><font size="2"><i>Рис. 4<br/> Демонстрация алгоритма Fast-Two-Sum, когда $a$ и $b$ имеют разный знак и $\left|b\right| > \frac{\left|a\right|}{2}$. $10010 + -1011 = 111 + 0$</i></font></center>

Проблема с использованием процедуры Fast-Two-Sum заключается в требовании $\left|a\right| \geqslant \left|b\right|$. Если это заранее неизвестно, то необходимо выполнить сравнение перед вызовом процедуры, что может потребовать относительно больших затрат по времени. Алгоритм Two-Sum, описанный ниже, избегает необходимости сравнения за счет трех дополнительных операций с плавающей точкой и работает быстрее в случае, когда отношение между $a$ и $b$ заранее не известно.

> **Теорема 7 (Knuth)** Пусть $a$ и $b$ — $p$-битные числа, причем $p \geqslant 3$. Тогда следующий алгоритм вернет неперекрывающееся разложение $x + y$ такое, что $a + b = x + y$, где $x$ - приближение суммы $a + b$, а $y$ представляет собой ошибку округления при вычислении $x$.

<div style="padding-left:100px">
Two-Sum (a, b)<br/>
1. $x \Leftarrow a \oplus b$<br/>
2. $b_{virtual} \Leftarrow x \ominus a$<br/>
3. $a_{virtual} \Leftarrow x \ominus b_{virtual}$<br/>
4. $b_{roundoff}\Leftarrow b \ominus b_{virtual}$<br/>
5. $a_{roundoff}\Leftarrow a \ominus a_{virtual}$<br/>
6. $y \Leftarrow a_{roundoff} \oplus b_{roundoff}$<br/>
7. return $(x, y)$<br/>
</div>


$\triangleright$
<div style="padding-left:50px">
Если $\left|a\right| \geqslant \left|b\right|$, то строки 1, 2 и 4 соответствуют строкам алгоритма Fast-Two-Sum. Вспомним из доказательства Теоремы 6, что строка 2 вычисляется точно. Следовательно, строка 3 вычисляется точно, так как $a_{virtual} = a$ может быть записано точно. Значит, $a_{roundoff} = 0$, $y = b_{roundoff}$ вычисляется точно и процедура корректна.
<br/>
Теперь предположим, что $\left|a\right| < \left|b\right|$, и рассмотрим два случая. Если  $\left|x\right| <\left|a\right| < \left|b\right|$, то $x$ вычисляется точно по лемме 4. Это означает, что $b_{virtual} = b, a_{virtual} = a$ и $b_{roundoff} = a_{roundoff} = y = 0$. 
<br/>
Во втором случае, если $\left|x\right| \geqslant \left|a\right|$, то строки 1 и 2 могут быть подвержены ошибке округления, и $x =  a + b + \mathrm{err}(a \oplus b)$ и $b_{virtual} = b + \mathrm{err}(a \oplus b) + \mathrm{err}(x \ominus a)$. Строки 2, 3 и 5 аналогичны строкам Fast-Two-Diff (строка 5 взята со знаком минус), следовательно строки 3 и 5 вычисляются точно. То есть $a_{virtual} = x - b_{virtual} = a - \mathrm{err}(x \ominus a)$ и $a_{roundoff} = \mathrm{err}(x \ominus a)$.
<br/>
Так как $\left|b\right| > \left|a\right|$, то $\left|x\right| = \left|a \oplus b\right| \leqslant 2\left|b\right|$ и ошибки округления $\mathrm{err}(a \oplus b)$ и $\mathrm{err}(x \ominus a)$ каждая из которых не может превышать $\mathrm{ulp}(b)$. Значит $b_{virtual} \in \left [ \frac{b}{2}, 2b \right ]$ и может быть применена лемма 5, чтобы показать точность вычислений в строке 4. Отсюда, $b_{roundoff} = - \mathrm{err}(a \oplus b) - \mathrm{err}(x \ominus a)$. Наконец, строка 6 точна из следствия 2 и поэтому число $a_{roundoff} + b_{roundoff} = - \mathrm{err}(a \oplus b)$ может быть представлено в $p$ битах.
<br/>
Итого, $y = - \mathrm{err}(a \oplus b)$ и $x = a + b + \mathrm{err}(a \oplus b)$, а значит $a + b = x + y$.
</div> 
$\triangleleft$
<br\>
Корректность алгоритма для вычитания доказывается аналогично.
<div style="padding-left:100px">
Two-Diff (a, b)<br/>
1. $x \Leftarrow a \ominus b$<br/>
2. $b_{virtual} \Leftarrow a \ominus x$<br/>
3. $a_{virtual} \Leftarrow x \oplus b_{virtual}$<br/>
4. $b_{roundoff}\Leftarrow b_{virtual} \ominus b$<br/>
5. $a_{roundoff}\Leftarrow a \ominus a_{virtual}$<br/>
6. $y \Leftarrow a_{roundoff} \oplus b_{roundoff}$<br/>
7. return $(x, y)$<br/>
    </div>

<center><img src="img/ts1.png"/></center>
<center><br><font size="2"><i>Рис. 5<br/> Демонстрация алгоритма Two-Sum, когда $\left|a\right| < \left|b\right|$ и $\left|a\right| \leqslant \left|x\right|$. $11.11 + 1101 = 10000 + 0.11$</i></font></center>

> **Следствие 8** Пусть $x$ и $y$ значения, которые вернули функции Fast-Two-Sum или Two-Sum.
Тогда 

>a) Если $\left|y\right| \geqslant 2^i$ для целого $i$, то $\left|x + y\right| \geqslant 2^i(2^p+1)$

>б) Если $\left|y\right| > 2^i$ для целого $i$, то $\left|x + y\right| > 2^i(2^{p+1}+1)$

$\triangleright$
<div style="padding-left:50px">
$y = -\mathrm{err}(a \oplus b)$ для некоторых $a$ и $b$. По теоремам 6 и 7, $a + b = x + y$. Требуемый результат напрямую следует из леммы 3.
</div>
$\triangleleft$

> **Следствие 9** Пусть $x$ и $y$ значения, которые вернули функции Fast-Two-Sum или Two-Sum.
Тогда, при использовании округления до ближайшего четного, $x$ и $y$ не смежные.

$\triangleright$
<div style="padding-left:50px">
Точное округление гарантирует, что $y \leqslant \frac{1}{2}\mathrm{ulp}(x)$. Если неравенство строгое, то $x$ и $y$ не смежные.
Если $y = \frac{1}{2}\mathrm{ulp}(x)$, то правило округления до ближайшего четного гарантирует, что наименее значимый бит мантиссы $x$ равен 0, следовательно $x$ и $y$ не смежные.
</div>
$\triangleleft$

## Суммирование разложений

Базируясь на алгоритмах сложения двух $p$-битных чисел, описанных выше, рассмотрим два алгоритма, позволяющих складывать два числа с произвольной точностью, представленных в виде разложений. Алгоритм Expansion-Sum выполняет сложение $m$-компонентного и $n$-компонентного чисел за время $O(mn)$, в то время как алгоритм Fast-Expansion-Sum делает это за время $O(n + m)$.

Несмотря на такое различие в асимптотике, первый алгоритм на практике может оказаться быстрее на разложениях, чей размер мал и фиксирован, потому что программные циклы могут быть полностью развернуты, а косвенные расходы времени исчезают, так как можно отказаться от использования массива. Линейный же алгоритм имеет определенные условия, при которых подобные оптимизации невозможны. 

Алгоритм Expansion-Sum имеет полезное свойство, что если входные данные были неперекрывающимися или несмежными, то выходные данные будут обладать тем же свойством.

Алгоритм Fast-Expansion-Sum имеет несколько недостатков. Первый из них — алгоритм не сохраняет свойств неперекрываемости/несмежности. Второй — алгоритм основывается на округлении до ближайшего четного, что делает его непереносимым.

Как правило, общим недостатком алгоритмов работы с разложениями является то, что в  разложении на выходе могут быть нулевые компоненты, даже если в исходном  разложении их не было. Например, если подать на вход  разложения $1111 + 0.0101$ и $1100 + 0.11$, то результатом будет $11100 + 0 + 0 + 0.0001$. К счастью, два данных алгоритма хорошо справляются с проблемой, если им на вход были переданы разложения с нулевыми компонентами.

Для начала рассмотрим алгоритм прибавления к разложению $p$-битного числа.

> **Теорема 10** Пусть $e = \sum^{m}_{i=1}e_i$ - неперекрывающееся $m$-компонентное  разложение; $b$ — $p$-битное число, где $p \geqslant 3$. Предполагается, что $e_1, e_2, \dots, e_m$ отсортированы в *возрастающем* по величине порядке, за исключением того, что любая компонента может равняться $0$. Тогда следующий алгоритм вернет неперекрывающееся разложение $h = \sum^{m + 1}_{i=1}h_i = e + b$, где компоненты $h_1, h_2, \dots, h_{m+1}$ также отсортированы в возрастающем порядке, за исключением того, что любая компонента может равняться $0$. Если разложение $e$ несмежное и используется округление к ближайшему четному, то разложение $h$ тоже несмежное.


<div style="padding-left:100px">
Grow-Expansion(e, b)<br/>
1. $Q_0 \Leftarrow b$<br/>
2. for $i \Leftarrow 1$ to $m$<br/>
3. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$(Q_i, h_i) \Leftarrow$ Two-Sum$(Q_{i-1}, e_i)$<br/>
4. $h_{m+1} \Leftarrow Q_m$<br/>
5. return $h$<br/>
</div>

<div style="padding-left:50px">$Q_i$ является приблизительной суммой $b$ и первых $i$ компонент разложения $e$.</div>


$\triangleright$
<div style="padding-left:50px">
В конце каждой итерации цикла $for$ сохраняется инвариант $Q_i + \sum_{j=1}^ih_j = b + \sum_{j=1}^ie_j$. Это очевидно верно для $i=0$ после исполнения строки 1. Из строки 3, применяя теорему 7, верно, что $Q_i + h_i = Q_{i-1} + e_i$. Отсюда по индукции утверждение верно для всех $i$. Таким образом, после исполнения строки 4, $\sum_{j=1}^{m+1}h_j = \sum_{j=1}^me_j + b$.<br/>

Для любого $i$ выходное значение операции Two-Sum в строке 3 обладает свойством, что $h_i$ и $Q_i$ не перекрываются. По лемме 1, $\left|h_i\right| \leqslant \left|e_i\right|$, и так как $e$ — неперекрывающееся разложение, чьи ненулевые компоненты расположены в возрастающем порядке, $h_i$ не может перекрываться ни с одним из $e_{i+1}, e_{i+2}, \dots$. Следовательно $h_i$ не может перекрываться с большими компонентами $h$, так как они получаются суммированием $Q_i$ с большими компонентами $e$. Отсюда $h$ — неперекрывающееся и возрастающее разложение(за исключением нулевых компонент). Если используется округление к ближайшему четному, то  $h_i$ и $Q_i$ — не смежные для любого $i$ по следствию 9. Значит, если $e$ — несмежное, то и $h$ — несмежное.<br/>

Если какое-либо $e_i$ равняется $0$, то соответствующая компонента $h_i$ равняется $0$ и значение аккумулятора $Q$ не меняется ($Q_i = Q_{i-1}$).
</div> 
$\triangleleft$

<center><img src="img/ge.png"/></center>
<center><br><font size="2"><i>Рис. 6<br/> Алгоритм Grow-Expansion. Разложения $e$ и $h$ имеют большие компоненты слева. Большее значение операции Two-Sum выходит влево, а меньшее вниз.</i></font></center>

> **Следствие 11** Каждая из первых $m$ компонет разложения $h$ не больше, чем соответствующая компонента разложения $e$. Кроме того $\left|h_1\right| \leqslant \left|b\right|$.

$\triangleright$
<div style="padding-left:50px">
Доказательство следует из непосредственного применения леммы 1 к строке 3.
</div> 
$\triangleleft$

> **Теорема 12** Пусть $e = \sum^{m}_{i=1}e_i$ и $f = \sum^{n}_{i=1}f_i$ - неперекрывающиеся $m$-компонентное и $n$-компонентное $p$-битные разложения, где $p \geqslant 3$. Предполагается, что компоненты $e$ и $f$ отсортированы в *возрастающем* по величине порядке, за исключением того, что любая компонента может равняться $0$. Тогда следующий алгоритм вернет неперекрывающееся разложение $h = \sum^{m + n}_{i=1}h_i = e + f$, где компоненты $h_1, h_2, \dots, h_{m+1}$ также отсортированы в возрастающем порядке, за исключением того, что любая компонента может равняться $0$. Если разложения $e$ и $f$ несмежные и используется округление к ближайшему четному, то разложение $h$ тоже несмежное.


<div style="padding-left:100px">
Expansion-Sum(e, f)<br/>
1. $h \Leftarrow e$<br/>
2. for $i \Leftarrow 1$ to $n$<br/>
3. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\langle h_i, h_{i+1}, \dots, h_{i+m}\rangle \Leftarrow$ Grow-Expansion$(\langle h_i, h_{i+1}, \dots, h_{i+m - 1}\rangle, f_i)$<br/>
4. return $h$<br/>
</div>

$\triangleright$
<div style="padding-left:50px">
Доказывается индукцией по строке 3.<br/>
После исполнения строки 1 алгоритм перебирает разложение $f$ от меньшей к большей компоненте и прибавляет к $h$, используя алгоритм Grow-Expansion. Когда рассматривается компонента $f_i$, компоненты $f_1, f_2, \dots, f_{i-1}$ уже прибавлены к $h$. Согласно следствию 11, $\left|h_j\right| \leqslant \left|f_j\right|$ после шага $j$ в строке 3. Так как $f$ — возрастающее неперекрывающееся разложение, то для любых $j < i$, $h_j$ не может перекрываться с $f_i$, и, кроме того, $\left|h_j\right| < \left|f_i\right|$ (если не $f_i=0$). Таким образом, прибавляя $f_i$ к $h$, можно пропустить $(i-1)$ компоненту разложения $h$, не потеряв свойств разложения $h$. Аналогично, если $e$ и $f$ несмежные, то можно пропустить первые $(i-1)$ компоненты $h$ без потери свойства несмежности разложения $h$. Если $f_i$ — нулевая компонента, то алгоритм Grow-Expansion выставит $h_i$ равным $0$ и продолжит исполнение, так как $0$ не перекрывается ни с каким числом.
</div> 
$\triangleleft$

<center><img src="img/es.png"/></center>
<center><br><font size="2"><i>Рис. 7<br/>Алгоритм Expansion-Sum</i></font></center>

В отличие от Expansion-Sum, Fast-Expansion-Sum не сохраняет свойства неперекрываемости и несмежности, но гарантирует, что если входные разложения были **строго неперекрывающимися**, то на выходе получится разложение с таким же свойством.

Разложение называется **строго неперекрывающимся**, если его компоненты попарно не перекрываются, ни одна компонента не смежна никаким двум другим, а также любая пара является степенями двойки.

Например, $11000 + 11$ и $10000 + 1000 + 10 + 1$ — строго неперекрывающиеся  разложения, а $11100 + 11$ и $100 + 10 + 1$ — нет.

Для разложения эта характеристика означает, что ноль в записи разложения должен появляться *как минимум* каждые $(p + 1)$ бит. Например разложение, каждая компонента которого есть $4$-битное число и максимальная величина которой равна $1111$, может быть не больше $1111.01111011110\dots$.

Каждое несмежное разложение является строго неперекрывающимся. Каждое строго неперекрывающееся разложение есть неперекрывающееся. В общем случае, обратное не верно. Так как любое число может быть представлено в виде несмежного разложения, то любое число имеет строго неперекрывающееся разложение.

Утверждается, что после предположения о том, что все разложения строго неперекрывающиеся, алгоритм, приведенный ниже, корректно работает на машинах с округлением до ближайшего четного.


> **Теорема 13** Пусть $e = \sum^{m}_{i=1}e_i$ и $f = \sum^{n}_{i=1}f_i$ - строго неперекрывающиеся $m$-компонентное и $n$-компонентное $p$-битные разложения, где $p \geqslant 4$. Предполагается, что компоненты $e$ и $f$ отсортированы в *возрастающем* по величине порядке, за исключением того, что любая компонента может равняться $0$. Тогда, на машинах с правилом округления к ближайшему четному, следующий алгоритм вернет строго неперекрывающееся разложение $h = \sum^{m + n}_{i=1}h_i = e + f$, где компоненты $h_1, h_2, \dots, h_{m+1}$ также отсортированы в возрастающем порядке, за исключением того, что любая компонента может равняться $0$.


<div style="padding-left:100px">
Fast-Expansion-Sum(e, f)<br/>
1. Объединить последовательности $e$ и $f$ в одну неубывающую последовательность $g$, возможно с нулевыми компонентами<br/>
2. $(Q_2, h_1) \Leftarrow $ Fast-Two-Sum$(g_2, g_1)$<br/>
3. for $i \Leftarrow 3$ to $m + n$<br/>
4. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$(Q_i, h_{i-1}) \Leftarrow$ Two-Sum$(Q_{i-1}, g_i)$<br/>
5. $h_{m+n} \Leftarrow Q_{m+n}$<br/>
6. return $h$<br/>
</div>

<div style="padding-left:50px">
$Q_i$ является приблизительной суммой первых $i$ компонент разложения $g$.<br/>
Для доказательства теоремы потребуется несколько лемм.<br/>
</div>

<center><img src="img/fes.png"/></center>
<center><br><font size="2"><i>Рис. 8<br/>Алгоритм Fast-Expansion-Sum</i></font></center>

> **Лемма 14 (Q invariant)** В конце каждой итерации цикла $for$ сохраняется инвариант $Q_i + \sum_{j=1}^{i-1}h_j = \sum_{j=1}^ig_j$. Это означает, что после исполнения строки 5, $\sum_{j=1}^{m+n}h_j = \sum_{j=1}^{m+n}g_j$, и алгоритм выдает корректную сумму.

$\triangleright$
<div style="padding-left:50px">
Инвариант верен для $i=2$ после исполнения строки 2. Строка 4 означает, что $Q_i + h_{i-1} = Q_{i-1} + g_i$. Отсюда, по индукции, утверждение верно для всех $i$.
</div> 
$\triangleleft$

> **Лемма 15** Пусть $\hat{g} = \sum_{j=1}^{k}\hat{g}_j$ — последовательность, сформированная из двух строго неперекрывающихся разложений. Предположим что $\hat{g}_k$ — наибольшая компонента и имеет ненулевой бит величины $2^i$ или меньше для некоторого $i$. Тогда $\left|\sum_{j=1}^{k}\hat{g}_j\right| < 2^i(2^{p+1}-1)$ и $\left|\sum_{j=1}^{k-1}\hat{g}_j\right| < 2^i(2^p)$.

$\triangleright$
<div style="padding-left:50px">
Пусть $\hat{e}$ и $\hat{f}$ — разложения, из которых получено $\hat{g}$. Предположим, что компонента $\hat{g}_k$ пришла из разложения $\hat{e}$. Так как $\hat{g}_k$ — наибольшая компонента $\hat{e}$ и имеет ненулевой бит величины $2^i$ или меньше, и $\hat{e}$ строго неперекрывающееся, то $\left|\hat{e}\right|$ меньше $2^i(2^p-\frac{1}{2})$. Например, при $p=4$ и $i=0$ верно, что $\left|\hat{e}\right| \leqslant 1111.0111101111\dots$. Такая же оценка верна для разложения $\hat{f}$, поэтому $\left|\hat{g}\right| = \left|\hat{e}+\hat{f}\right| < 2^i(2^{p+1}-1)$.<br/>

Если убрать $\hat{g}_k$ из суммы, то надо рассмотреть два случая. Если $\hat{g}_k=2^i$, то $\left|\hat{e}-\hat{g}_k\right|$ меньше чем $2^i(2^p-2)$ и $\left|\hat{f}\right|$ меньше чем $2^i(2)$. Обратно, если $\hat{g}_k\neq 2^i$, то $\left|\hat{e}-\hat{g}_k\right|$ меньше чем $2^i(\frac{1}{2})$ и $\left|\hat{f}\right|$ меньше чем $2^i(2^p - \frac{1}{2})$. В любом случае, $\left|\hat{g}-\hat{g}_k\right|=\left|\hat{e}-\hat{g}_k+\hat{f}\right|\leqslant2^i(2^p)$. 


</div> 
$\triangleleft$

> **Лемма 16** Разложение $h$, полученное из алгоритма Fast-Expansion-Sum, является неперекрывающимся, и его компоненты расположены по возрастанию величины (исключая нули).

$\triangleright$
<div style="padding-left:50px">
Предположим от противного, что две последовательные ненулевые компоненты $h$ перекрываются или идут в убывающем порядке. Обозначим такую пару $h_{i-1}$ и $h_i$. Тогда компоненты $h_1, h_2, \dots, h_{i-1}$ не перекрываются и возрастают (исключая нулевые).<br/>

Не уменьшая общности предположим, что экспонента $h_{i-1}$ равна нулю, то есть $h_{i-1}$ может быть записано в форме $\pm 1.*$, где звездочка заменяет последовательность битов.<br/>

$Q_i$ и $h_{i-1}$ получены в результате алгоритма Two-Sum или Fast-Two-Sum, поэтому они не смежны по следствию 9 (используется округление к ближайшему четному). $Q_i$ представимо в форме $\pm *00$ (нет бит меньше 4 по абсолютной величине). Так как $\left|h_{i-1}\right|\geqslant 1$, то следствие 8а) гарантирует, что $\left|Q_i + h_{i-1}\right|\geqslant 2^p+1$.<b>(1)</b><br/>

Так как компоненты $h_{i-1}$ и $h_{i}$ ненулевые, а также перекрываются или расположены в убывающем порядке, то в мантиссе $h_{i}$ должен быть хотя бы один ненулевой бит, чья величина не больше $1$. Откуда появляется этот бит? Поскольку $h_{i}$ вычисляется в строе 4 из $Q_i$ и $g_{i+1}$, то подозрительный бит не мог прийти из $Q_i$ из-за его формы, поэтому он пришел из $g_{i+1}$. Отсюда $\left|g_{i+1}\right|$ имеет ненулевой бит величины $1$ или меньше. Применяя лемму 15, получаем $\left|\sum_{j=1}^{i}g_j\right| < 2^p$.<br/>

Вспоминая форму $h_{i-1}$ и факт, что $h_1, h_2, \dots, h_{i-1}$ не перекрываются и возрастают, получаем, что $\left|\sum_{j=1}^{i-2}h_j\right| < 1$.<br/>

Переписав лемму 14 в виде $Q_i + h_{i-1} = \sum_{j=1}^ig_j - \sum_{j=1}^{i-2}h_j$, приходим к выводу, что $\left|Q_i + h_{i-1}\right| < 2^p + 1$.<b>(2)</b><br/>

Получили противоречие из (1) и (2), следовательно лемма верна.<br/>
</div> 
$\triangleleft$

#### Доказательство Теоремы 13

$\triangleright$
<div style="padding-left:50px">
Лемма 14 гарантирует, что $h = e + f$. Лемма 16 исключает для компонент $h$ возможность перекрываться или располагаться в убывающем порядке. Остается доказать, что разложение $h$ строго не перекрывается. Предположим, что две последовательные ненулевые компоненты $h_{i-1}$ и $h_i$ являются смежными.<br/>

Не уменьшая общности предположим, что экспонента $h_{i-1}$ равна нулю, то есть $h_{i-1}$ может быть записано в форме $\pm 1.*$,  где звездочка заменяет последовательность битов. Как и в лемме 16, $Q_i$ представимо в форме $\pm *00$ (нет бит меньше 4 по абсолютной величине).<br/>

Так как $h_{i-1}$ и $h_i$ являются смежными, наименьший ненулевой бит $h_i$ имеет величину $2$, то есть $h_i$  представим в виде $\pm *10$. Снова зададимся вопросом, откуда пришел этот бит. Как и в предыдущей лемме, он не мог прийти из $Q_i$, поэтому должен был прийти из $g_{i+1}$. Отсюда $\left|g_{i+1}\right|$ имеет ненулевой бит величины $2$. Применяя лемму 15, получаем $\left|\sum_{j=1}^{i+1}g_j\right| < 2^{p+2}-2$ и $\left|\sum_{j=1}^{i}g_j\right| < 2^{p+1}$.<br/>

Вспоминая форму $h_{i-1}$ и тот факт, что $h_1, h_2, \dots, h_{i-1}$ не перекрываются и возрастают, получаем, что $\left|\sum_{j=1}^{i-1}h_j\right| < 2$ и $\left|\sum_{j=1}^{i-2}h_j\right| < 1$.<br/>

Переписав лемму 14 в виде $Q_{i+1} + h_{i} = \sum_{j=1}^{i+1}g_j - \sum_{j=1}^{i-1}h_j$, приходим к выводу, что $\left|Q_{i+1} + h_i\right| < 2^{p+2}$.<b>(3)</b><br/>

Переписав лемму 14 в виде $Q_i + h_{i-1} = \sum_{j=1}^ig_j - \sum_{j=1}^{i-2}h_j$, приходим к выводу, что $\left|Q_i + h_{i-1}\right| < 2^{p+1} + 1$.<b>(4)</b><br/>

Вспомним, что $\left|h_i\right| \geqslant 2$. Предположив, что $\left|h_i\right| > 2$, из следствия 8б) получаем, что это возможно только при $\left|Q_{i+1} + h_i\right|> 2^{p+2}+2$, что противоречит неравенству 3. Значит $\left|h_i\right| = 2$ и выражается в одном бите.<br/>

Аналогично,  $\left|h_{i-1}\right| \geqslant 1$. Предположив, что $\left|h_{i-1}\right| > 1$, из следствия 8б) получаем, что это возможно только при $\left|Q_{i} + h_{i-1}\right|> 2^{p+1}+1$, что противоречит неравенству 4. Значит $\left|h_{i-1}\right| = 1$ и выражается в одном бите.<br/>

Из следствия 8а) $\left|Q_{i} + h_{i-1}\right|\geqslant 2^{p}+1$ (так как $\left|h_{i-1}\right| = 1$). Используя факт $\left|\sum_{j=1}^{i-2}h_j\right| < 1$ и лемму 14, получаем $\left|\sum_{j=1}^{i}g_j\right| > 2^p$. Так как $g$ получен из двух неперекрывающихся возрастающих разложений, то $\left|g_i\right| \geqslant 2^{p-2} \geqslant 100_2$ (из-за $p \geqslant 4$), следовательно $g_{i+2}, g_{i+3}, \dots$ должны быть в форме $\pm *000$. $Q_{i+1}$ также в форме $\pm *000$, так как $Q_{i+1}$ и $h_{i}$ получены из Two-Sum или Fast-Two-Sum, а значит не смежны по следствию 9.<br/>

Используя $Q_{i+1}$ и $g_{i+2}, g_{i+3}, \dots$ в форме $\pm *000$, получаем что $h_{i+1}, h_{i+2}, \dots$ также в форме $\pm *000$ и, следовательно, не смежны с $h_i$. Значит $h$ не может содержать трех последовательных смежных компонент.<br/>

Это доказывает, что если две компоненты $h$ смежны, то обе выражаются в одном бите и больше не смежны ни с одной из оставшихся, то есть $h$ строго неперекрывающееся разложение.
</div> 
$\triangleleft$

<center><img src="img/fes2.png"/></center>
<center><br><font size="2"><i>Рис. 9<br/>4-битовый пример алгоритма Fast-Expansion-Sum. $11110.1 + 10011110.1 = (11110 + 0.1) + (10000000 + 11110 + 0.1) = 11000000 + 0 + -10 + -1 + 0$</i></font></center>

## Простое умножение

Алгоритм простого умножения вычисляет непересекающееся разложение, равное произведению двух $p$-битных чисел. Вычисление производится с помощью разделения каждого числа на две части с половинной точностью и выполнения четырех точных умножений над этими частями.

> **Теорема 17** Пусть $a$ — $p$-битное число с плавающей точкой при $p \geqslant 3$. Выберем **точку разбиения** $s$, такую что $\frac{p}{2}\leqslant s \leqslant p-1$. Тогда следующий алгоритм вернет $(p-s)$-битное число $a_{hi}$ и $(s-1)$-битное число $a_{lo}$, такие что $\left|a_{hi}\right| \geqslant \left|a_{lo}\right|$ и $a = a_{hi} + a_{lo}$.

<div style="padding-left:100px">
Split(a, s)<br/>
1. $c \Leftarrow (2^s + 1)\otimes a$<br/>
2. $a_{big} \Leftarrow c\ominus a$<br/>
3. $a_{hi} \Leftarrow c\ominus a_{big}$<br/>
4. $a_{lo} \Leftarrow a\ominus a_{hi}$<br/>
5. return $(a_{hi}, a_{lo})$<br/>
</div>

$\triangleright$
<div style="padding-left:50px">
Строка 1 эквивалентна вычислению $2^sa\oplus a$ ($2^sa$ может быть записано точно, так как умножение на степень двойки меняет только экспоненту и не меняет мантиссу числа). Строка 1 подвержена ошибке округления, поэтому $c = 2^sa + a + \mathrm{err}(2^sa\oplus a)$.<br/>

Строка 2 также подвержена ошибке округления, поэтому $a_{big} = 2^sa + \mathrm{err}(2^sa\oplus a) + \mathrm{err}(c\ominus a)$. Необходимо показать, что экспонента числа $a_{big}$ не превосходит экспоненты $2^sa$. Каждая из ошибок округления $\left|\mathrm{err}(2^sa\oplus a)\right|$ и $\left|\mathrm{err}(c\ominus a)\right|$ не превосходит $\frac{1}{2}\mathrm{ulp}(c)$, поэтому экспонента числа $a_{big}$ может быть больше экспоненты $2^sa$, только если каждый бит мантиссы числа $a$, кроме возможно последнего, ненулевой. Можно вручную рассмотреть эти два случая и удостовериться, что экспонента числа $a_{big}$ никогда не превосходит экспоненты $2^sa$.<br/>

По лемме 5, строки 3 и 4 вычисляются точно, поэтому $a_{hi} = a - \mathrm{err}(c\ominus a)$ и $a_{lo} = \mathrm{err}(c\ominus a)$. Так как $\left|\mathrm{err}(c\ominus a)\right|\leqslant\frac{1}{2}\mathrm{ulp}(2^sa)$, то $a_{lo}$ выражается в $(s-1)$ битах. Наименьший значащий бит $a_{hi}$ не может быть меньше чем $\mathrm{ulp}(a_{big}) = 2^s\mathrm{ulp}(a)$. Если $a_{hi}$ имеет такую же экспоненту как $a$, то он выражается в $(p-s)$ битах. Иначе, если $a_{hi}$ имеет экспоненту большую чем $a$ (потому что $a - \mathrm{err}(c\ominus a)$ имеет большую экспоненту чем $a$), то $a_{hi}$ выражается в $1$ бите.<br/>

Наконец, из точного вычисления строки 4 следует, что $a = a_{hi} + a_{lo}$.
</div> 
$\triangleleft$

<center><img src="img/split.png"/></center>
<center><br><font size="2"><i>Рис. 10<br/>Пример алгоритма Split, выполняющего разбиение $5$-битового числа на два $2$-битовых.$11101 = 10000 + -11$</i></font></center>

Умножение двух чисел выполняется с помощью выбора $s = \left \lceil \frac{p}{2} \right \rceil$, так что каждое из $p$-битовых чисел $a$ и $b$ разбивается на две $\left \lfloor \frac{p}{2} \right \rfloor$-битовые части: $a_{hi}, a_{lo}, b_{hi}, b_{lo}$. Произведения $a_{hi}\times b_{hi}, a_{lo}\times b_{hi}, a_{hi}\times b_{lo}, a_{lo}\times b_{lo}$ могут быть вычислены точно и затем просуммированы с помощью алгоритма Fast-Expansion-Sum. Однако существует способ выполнить эти вычисления быстрее.

> **Теорема 18** Пусть $a$ и $b$ есть $p$-битные числа, причем $p \geqslant 6$. Тогда следующий алгоритм вернет неперекрывающееся разложение $x + y$ такое, что $a \times b = x + y$, где $x$ - приближение произведения $a \times b$, а $y$ представляет собой ошибку округления при вычислении $x$. Кроме того, если используется правило округления до ближайшего четного, то $x$ и $y$ несмежные.

<div style="padding-left:100px">
Two-Product(a, b)<br/>
1. $x \Leftarrow a\otimes b$<br/>
2. $(a_{hi}, a_{lo}) =$ Split$(a, \left \lceil \frac{p}{2} \right \rceil)$<br/>
3. $(b_{hi}, b_{lo}) =$ Split$(b, \left \lceil \frac{p}{2} \right \rceil)$<br/>
4. $err_1 \Leftarrow x\ominus (a_{hi}\otimes b_{hi})$<br/>
5. $err_2 \Leftarrow err_1\ominus (a_{lo}\otimes b_{hi})$<br/>
6. $err_3 \Leftarrow err_2\ominus (a_{hi}\otimes b_{lo})$<br/>
7. $y \Leftarrow (a_{lo}\otimes b_{lo}) \ominus err_3$<br/>
8. return $(x, y)$<br/>
</div>

$\triangleright$
<div style="padding-left:50px">
Строка 1 подвержена ошибке округления, поэтому $x = a\times b + \mathrm{err}(a\otimes b)$. Умножения в строках 4-7 точные, так как каждый множитель имеет не более $\left \lfloor \frac{p}{2} \right \rfloor$ бит. Если доказать, что вычитания точны, то $y = -\mathrm{err}(a\otimes b)$.<br/>

Не уменьшая общности предположим, что экспоненты чисел $a$ и $b$ равны, поэтому числа $\left|a\right|$ и $\left|b\right|$ находятся в интервале $[2^{p-1}, 2^p-1]$. Из доказательства теоремы 17 следует, что $\left|a_{hi}\right|$ и $\left|b_{hi}\right|$ находятся в интервале $[2^{p-1}, 2^p]$, а $\left|a_{lo}\right|$ и $\left|b_{lo}\right|$ находятся в интервале $[0, 2^{\left \lceil \frac{p}{2} \right \rceil - 1}]$. Из этих оценок и условия $p\geqslant 6$ получаем $\left|a_{lo}\right|\leqslant\frac{1}{8}\left|a_{hi}\right|$, $\left|b_{lo}\right|\leqslant\frac{1}{8}\left|b_{hi}\right|$ и $\mathrm{err}(a\otimes b)\leqslant2^{p-1}\leqslant\frac{1}{32}\left|a_{hi}\times b_{hi}\right|$.<br/>

Заметим, что $x = a\times b + \mathrm{err}(a\otimes b) = a_{hi}\times b_{hi} + a_{lo}\times b_{hi} + a_{hi}\times b_{lo} + a_{lo}\times b_{lo} + \mathrm{err}(a\otimes b) = a_{hi}\times b_{hi} \pm \frac{19}{64}\left|a_{hi}\times b_{hi}\right|$ из неравенств выше. Применяя лемму 5, получим точное вычитание, а значит и точную строку 4, поэтому $\mathrm{err}_1 = a_{lo}\times b_{hi} + a_{hi}\times b_{lo} + a_{lo}\times b_{lo} + \mathrm{err}(a\otimes b)$.<br/>

Строка 5 вычисляется точно, если значение $\mathrm{err}_1 - a_{lo}\times b_{hi} = a_{hi}\times b_{lo} + a_{lo}\times b_{lo} + \mathrm{err}(a\otimes b)$ выражается в $p$ битах. Это доказывается, если показать, что левая часть — это множитель $2^{\left \lceil \frac{p}{2} \right \rceil}$, а правая часть строго меньше чем $2^{\left \lceil \frac{3p}{2} \right \rceil}$.<br/>

Верхняя граница абсолютного значения правой части следует из оценок $a_{hi}, a_{lo}, b_{lo}$ и $\mathrm{err}(a\otimes b)$. Чтобы показать, что левая часть является множителем $2^{\left \lceil \frac{p}{2} \right \rceil}$, вспомним что $\mathrm{err}_1$ — это множитель $2^{p-1}$, так как $a\otimes b$ и $a_{hi}\times b_{hi}$ имеют экспоненты хотя бы $2p-2$.<br/>

Значит $\mathrm{err}_1 - a_{lo}b_{hi}$ должен быть множителем $2^{\left \lceil \frac{p}{2} \right \rceil}$, потому что $a_{lo}$ — целое, а $b_{hi}$ — это множитель $2^{\left \lceil \frac{p}{2} \right \rceil}$. Итого, строка 5 вычисляется точно и $\mathrm{err}_2 = a_{hi}\times b_{lo} + a_{lo}\times b_{lo} + \mathrm{err}(a\otimes b)$.<br/>

Строка 6 вычисляется точно, потому что $a_{lo}\times b_{lo}$ является целым, не превосходящим $2^{p-1}$ (так как $a_{lo}$ и $b_{lo}$ — целые не больше $2^{\left \lceil \frac{p}{2} \right \rceil-1}$). Отсюда, $\mathrm{err}(a\otimes b)$ — целое не больше $2^{p-1}$, поэтому $\mathrm{err}_3 = a_{lo}\times b_{lo} + \mathrm{err}(a\otimes b)$ — целое не больше $2^p$ и выражается в $p$ битах.<br/>

Строка 7 вычисляется точно, так как $y = -\mathrm{err}(a\otimes b)$ выражается в $p$ битах. Значит, $a\times b = x + y$.<br/>

Если применяется округление к ближайшему четному, то $x$ и $y$ — несмежные по аналогии со следствием 9.<br/>
</div> 
$\triangleleft$

<center><img src="img/tp.png"/></center>
<center><br><font size="2"><i>Рис. 11<br/>Пример алгоритма Two-Product в 6-битовой арифметике, где $a = b = 111011$, $a_{hi} = b_{hi} = 111000$, $a_{lo} = b_{lo} = 11$. Каждое промежуточное вычисление выражается в 6 битах. Итоговое выражение равно $110110\times2^6 + 11001$.</i></font></center>